In [0]:
from pyspark.sql.functions import *
import json

In [0]:
tables=["accounts","branches","customers","transactions"]
for table in tables:
    df=spark.read.table("dev.bronze.raw_"+table).dropDuplicates()
    df=df.drop("ingestion_date","path")
    df.write.mode("overwrite").saveAsTable("dev.silver.silver_"+table)

In [0]:
%sql
select * from dev.silver.silver_customers;

In [0]:
df = spark.sql(
    """
    SELECT
        a.customer_id,a.first_name,a.last_name,a.date_of_birth,a.gender,a.city as cust_city,a.state cust_state,a.phone,a.email as cust_email
        ,b.account_id,b.account_number,b.account_status,b.account_type,b.available_balance,b.current_balance,b.interest_rate,b.is_joint_account,c.*,d.*
    FROM dev.silver.silver_customers a
    LEFT JOIN dev.silver.silver_accounts b
        ON a.customer_id = b.customer_id
    LEFT JOIN dev.silver.silver_branches c
        ON b.branch_code = c.branch_code
    LEFT JOIN dev.silver.silver_transactions d
        ON b.account_id = d.from_account_id
    """
)
df.write.mode("overwrite").saveAsTable("dev.silver.silver_customer_account_txn")

In [0]:
%sql
select * from dev.silver.silver_customer_account_txn